In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn
import skill_metrics as sm
import torch
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader

from src.dataset import CERDataset
from src.mlp import MLPLucas
from src.saets import AutoEncoder, FinalModule

seaborn.set()
torch.manual_seed(32)
np.random.seed(32)

HORIZONS = 12
WINDOW = 3
FORWARD_EXPANSION = 1
N_LAYERS = 1
DROPOUT = 0.0
DECOMP_METHOD = 'fft'
COMPLEXO_EOLICO='Amontada'
CENTRAL_EOLICA='BC'
TIME_STEP='1_mon'
MES='Dec'
DECOMP=True
DEVICE = torch.device('cuda')


In [2]:
if not os.path.exists(f'data/components/{CENTRAL_EOLICA}_{MES}_{TIME_STEP}'):
    os.system(f'mkdir data/components/{CENTRAL_EOLICA}_{MES}_{TIME_STEP}')
if not os.path.exists(f'data/out/{CENTRAL_EOLICA}_{MES}_{TIME_STEP}'):
    os.system(f'mkdir data/out/{CENTRAL_EOLICA}_{MES}_{TIME_STEP}')

In [3]:
def plot_taylor(refs: dict, predictions_dict: dict):

    models = list(predictions_dict.keys())
    colors = ['c', 'm', 'y', 'k', 'r', 'b', 'g']
    colors = colors[:len(models)]
    models = {model: color for model, color in zip(models, colors)}
    for idx, (model, pred_dict) in enumerate(predictions_dict.items()):
        taylor_stats = []
        name = model[0]
        if model.endswith('ND'):
            name = name + 'ND'
        for horizon, pred in pred_dict.items():
            taylor_stats.append(sm.taylor_statistics(pred, refs[name][int(horizon)], 'data'))

        sdev = np.array([taylor_stats[0]['sdev'][0]]+[x['sdev'][1]
                                                    for x in taylor_stats])
        crmsd = np.array([taylor_stats[0]['crmsd'][0]]+[x['crmsd'][1]
                                                        for x in taylor_stats])
        ccoef = np.array([taylor_stats[0]['ccoef'][0]]+[x['ccoef'][1]
                                                        for x in taylor_stats])

        # To change other params in the plot, check SkillMetrics documentation in
        # https://github.com/PeterRochford/SkillMetrics/wiki/Target-Diagram-Options
        if len(list(predictions_dict.keys())) != 1:
            if idx != len(list(predictions_dict.keys()))-1 or len(list(predictions_dict.keys())) == 1:
                sm.taylor_diagram(sdev, crmsd, ccoef, styleOBS='-',
                                colOBS='g', markerobs='o',
                                titleOBS='Observation',
                                markercolor=models[model])
            else:
                sm.taylor_diagram(sdev, crmsd, ccoef, styleOBS='-',
                                titleOBS='Observation',
                                colOBS='g', markerobs='o', markercolor=models[model],
                                overlay = 'on', markerLabel=models)
        else:
            sm.taylor_diagram(sdev, crmsd, ccoef, styleOBS='-',
                      colOBS='g', markerobs='o',
                      titleOBS='Observation', markercolor='c',
                      markerLabel=['placeholder']+[
                          k+1 for k, v in pred_dict.items()])

In [4]:
dataset = CERDataset(window=WINDOW,
                     horizons=HORIZONS,
                     complexo_eolico=COMPLEXO_EOLICO,
                     central_eolica=CENTRAL_EOLICA,
                     time_step=TIME_STEP,
                     mes=MES,
                     decomp=DECOMP,
                     decomp_method=DECOMP_METHOD)

train_loader = DataLoader(dataset, batch_size=128,
                          shuffle=True, num_workers=8)


aaaaaaaaa [  4   7   9  14  19  21  26  29  33  36  41  43  46  48  52  54  57  59
  62  64  67  70  73  78  80  82  85  87  90  94  96 100 104 107 109 113
 119 125 127 129 134 136 138 141 144 146 149 153 156 161 163 166 169 173
 175 179 181 184 187 190 192 194 199 202 204 206 208 210 212 216 222 224
 227 229 232 234 236 238 241]
Decomp serie in 6 components
Getting test components


/home/mateus/miniconda3/envs/ts/lib/python3.8/site-packages/numpy/core/_asarray.py:102: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [5]:
input_example = next(iter(train_loader))[0]
input_size = input_example.shape[1]*input_example.shape[2]

In [6]:
auto_encoder = AutoEncoder(input_size=input_size,
                               horizons=HORIZONS, device=DEVICE,
                               forward_expansion=FORWARD_EXPANSION,
                               num_layers=N_LAYERS,
                               dropout=DROPOUT)

trainer = Trainer(gpus=1, max_epochs=5)
trainer.fit(auto_encoder, train_dataloader=train_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
/home/mateus/miniconda3/envs/ts/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: you defined a validation_step but have no val_dataloader. Skipping val loop
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 812 K 
1 | decoder | Decoder | 1.1 M 
------------------------------------
1.9 M     Trainable params
0         Non-trainable params
1.9 M     Total params
7.612     Total estimated model params size (MB)


Epoch 2: 100%|██████████| 4/4 [00:00<00:00,  5.34it/s, loss=0.536, v_num=32, train_loss_step=0.150, train_loss_epoch=0.362] 

/home/mateus/miniconda3/envs/ts/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


In [ ]:
final_model = FinalModule(input_size=input_size,
                              horizons=HORIZONS, device=DEVICE,
                              forward_expansion=FORWARD_EXPANSION,
                              num_layers=N_LAYERS,
                              dropout=DROPOUT)
final_model.load_encoder(auto_encoder.encoder)
trainer = Trainer(gpus=1, max_epochs=5)
trainer.fit(final_model, train_dataloader=train_loader)

In [ ]:
dataset_lucas = CERDataset(window=WINDOW,
                           horizons=HORIZONS,
                           complexo_eolico=COMPLEXO_EOLICO,
                           central_eolica=CENTRAL_EOLICA,
                           time_step=TIME_STEP,
                           mes=MES,
                           decomp=DECOMP,
                           decomp_method='2fft')

train_loader_lucas = DataLoader(dataset_lucas, batch_size=128,
                                shuffle=True, num_workers=8)
input_example_lucas = next(iter(train_loader_lucas))[0]
input_size_lucas = input_example_lucas.shape[1] * \
    input_example_lucas.shape[2]

mlp = MLPLucas(window_size=input_example_lucas.shape[1],
               n_comps=input_example_lucas.shape[2],
               horizons=HORIZONS)
trainer = Trainer(gpus=1, max_epochs=5)
trainer.fit(mlp, train_dataloader=train_loader_lucas)


In [ ]:
dataset.set_type('test')
dataset_lucas.set_type('test')
mlp = mlp.cpu()
final_model = final_model.cpu()
X_test = dataset.samples
X_test_lucas = dataset_lucas.samples
y = dataset.labels
y_final = final_model(X_test).detach()/dataset.test_scaler.scale_
y_mlp = mlp(X_test_lucas).detach()/dataset_lucas.test_scaler.scale_

y_final = y_final.numpy()
y_mlp = y_mlp.numpy()
y = y.numpy()/dataset.test_scaler.scale_
preds = {}
preds['SAETS'] = {i: y_final[:, i] for i in range(HORIZONS)}
preds['Cabral'] = {i: y_mlp[:, i] for i in range(HORIZONS)}
refs = {key: {i: y[:, i] for i in range(HORIZONS)} for key in ['S', 'C']}

In [ ]:
plot_taylor(refs, preds)
plt.show()

plt.plot(y[:, 0])
plt.plot(y_final[:, 0])
plt.plot(y_mlp[:, 0])
plt.legend(['Original', 'SAETS', 'Cabral'])
plt.show()